In [1]:
import torch
import torch.nn as nn
import numpy as np
import math
import random
import os
import re

"""
基于pytorch的only decoder语言模型
"""
import torch
import torch.nn as nn
import math
import torch.nn.functional as F
import copy

from new_nnbert import Encoder

#加载字表
def build_vocab(vocab_path):
    vocab = {"<pad>":0}
    with open(vocab_path, encoding="utf8") as f:
        for index, line in enumerate(f):
            char = line[:-1]       #去掉结尾换行符
            vocab[char] = index + 1 #留出0位给pad token
    return vocab

#加载语料
def load_corpus(path):
    corpus = ""
    with open(path, encoding="gbk") as f:
        for line in f:
            corpus += line.strip()
    return corpus

#随机生成一个样本
#从文本中截取随机窗口，前n个字作为输入，最后一个字作为输出
def build_sample(vocab, window_size, corpus):
    start = random.randint(0, len(corpus) - 1 - window_size)
    end = start + window_size
    window = corpus[start:end]
    target = corpus[start + 1:end + 1]  #输入输出错开一位
    # print(window, target)
    x = [vocab.get(word, vocab["<UNK>"]) for word in window]   #将字转换成序号
    y = [vocab.get(word, vocab["<UNK>"]) for word in target]
    return x, y

#建立数据集
#sample_length 输入需要的样本数量。需要多少生成多少
#vocab 词表
#window_size 样本长度
#corpus 语料字符串
def build_dataset(sample_length, vocab, window_size, corpus):
    dataset_x = []
    dataset_y = []
    for i in range(sample_length):
        x, y = build_sample(vocab, window_size, corpus)
        dataset_x.append(x)
        dataset_y.append(y)
    return torch.LongTensor(dataset_x), torch.LongTensor(dataset_y)

#建立模型
def build_model(vocab, char_dim):
    # model = LanguageModel(char_dim, vocab)
    model = Encoder(vocab, char_dim, 2, 12)
    return model

#文本生成测试代码
def generate_sentence(openings, model, vocab, window_size):
    reverse_vocab = dict((y, x) for x, y in vocab.items())
    model.eval()
    src_mask = torch.tril(torch.ones(window_size, window_size)).bool().cuda()
    with torch.no_grad():
        pred_char = ""
        #生成了换行符，或生成文本超过30字则终止迭代
        while pred_char != "\n" and len(openings) <= 30:
            openings += pred_char
            x = [vocab.get(char, vocab["<UNK>"]) for char in openings[-window_size:]]
            x = torch.LongTensor([x])
            if torch.cuda.is_available():
                x = x.cuda()
            
            y = model(x, y=None, mask=src_mask)[0]
            # print(y.shape)
            index = sampling_strategy(y[-1])
            pred_char = reverse_vocab[index]
    return openings

def sampling_strategy(prob_distribution):
    if random.random() > 0.1:
        strategy = "greedy"
    else:
        strategy = "sampling"
    if strategy == "greedy":
        return int(torch.argmax(prob_distribution))
    elif strategy == "sampling":
        prob_distribution = prob_distribution.cpu().numpy()
        return np.random.choice(list(range(len(prob_distribution))), p=prob_distribution)


#计算文本ppl
def calc_perplexity(sentence, model, vocab, window_size):
    prob = 0
    model.eval()
    with torch.no_grad():
        for i in range(1, len(sentence)):
            start = max(0, i - window_size)
            window = sentence[start:i]
            x = [vocab.get(char, vocab["<UNK>"]) for char in window]
            x = torch.LongTensor([x])
            target = sentence[i]
            target_index = vocab.get(target, vocab["<UNK>"])
            if torch.cuda.is_available():
                x = x.cuda()
            pred_prob_distribute = model(x)[0][-1]
            target_prob = pred_prob_distribute[target_index]
            prob += math.log(target_prob, 10)
    return 2 ** (prob * ( -1 / len(sentence)))


def train(corpus_path, save_weight=False):
    epoch_num = 50        #训练轮数
    batch_size = 64       #每次训练样本个数
    train_sample = 50000  #每轮训练总共训练的样本总数
    char_dim = 768        #每个字的维度
    window_size = 10       #样本文本长度
    vocab = build_vocab("vocab.txt")       #建立字表
    corpus = load_corpus(corpus_path)     #加载语料
    model = build_model(vocab, char_dim)    #建立模型
    src_mask = torch.tril(torch.ones(window_size, window_size)).bool().cuda()
    # print(model)
    if torch.cuda.is_available():
        model = model.cuda()
    optim = torch.optim.Adam(model.parameters(), lr=0.001)   #建立优化器
    print("文本词表模型加载完毕，开始训练")
    for epoch in range(epoch_num):
        print(f'当前epoch:{epoch}')
        model.train()
        watch_loss = []
        for batch in range(int(train_sample / batch_size)):
            x, y = build_dataset(batch_size, vocab, window_size, corpus) #构建一组训练样本
            # print(x.shape, y.shape)
            if torch.cuda.is_available():
                x, y = x.cuda(), y.cuda()
            optim.zero_grad()    #梯度归零
            loss = model(x, y, src_mask)   #计算loss
            # print(loss.item())
            loss.backward()      #计算梯度
            optim.step()         #更新权重
            watch_loss.append(loss.item())
        print("=========\n第%d轮平均loss:%f" % (epoch + 1, np.mean(watch_loss)))
        print(generate_sentence("让他在半年之前，就不能做出", model, vocab, window_size))
        print(generate_sentence("李慕站在山路上，深深的呼吸", model, vocab, window_size))
    if not save_weight:
        return
    else:
        base_name = os.path.basename(corpus_path).replace("txt", "pth")
        model_path = os.path.join("model", base_name)
        torch.save(model.state_dict(), model_path)
        return
if __name__ == "__main__":
    # build_vocab_from_corpus("corpus/all.txt")
    train("corpus.txt", False)

文本词表模型加载完毕，开始训练
当前epoch:0
第1轮平均loss:4.467230
让他在半年之前，就不能做出来，是他是不能在这里，他每一个月，他
李慕站在山路上，深深的呼吸了一个人，但他们对于他们的，他们的身
当前epoch:1
第2轮平均loss:4.117742
让他在半年之前，就不能做出来安全，但这些日子，李慕的身体，李慕
李慕站在山路上，深深的呼吸声，李慕的身体，李慕就是一个月，但也
当前epoch:2
第3轮平均loss:4.032398
让他在半年之前，就不能做出了一个人，他们和李慕，说道：“你们不
李慕站在山路上，深深的呼吸口气，逐渐的消耗，他们出现在他的身体
当前epoch:3
第4轮平均loss:3.989936
让他在半年之前，就不能做出了一些，他们的事情，他们的，他们的身
李慕站在山路上，深深的呼吸收了一些，身体，也不能再次出现在的，
当前epoch:4
第5轮平均loss:3.945637
让他在半年之前，就不能做出步。李慕看到李慕，说道：“你们不是我
李慕站在山路上，深深的呼吸，说道：“苏！”李慕道：“你们不知道
当前epoch:5
第6轮平均loss:3.906239
让他在半年之前，就不能做出的事情，他。”李慕道：“你们知道姐姐
李慕站在山路上，深深的呼吸风，说道：“你们是我们的，那么多好，
当前epoch:6
第7轮平均loss:3.866904
让他在半年之前，就不能做出这么快，他们的，李慕不是一个人，不是
李慕站在山路上，深深的呼吸，一声，说道：“你们通过这种事件事情
当前epoch:7
第8轮平均loss:3.832428
让他在半年之前，就不能做出了这么多，他们也也按照她们的的实力，
李慕站在山路上，深深的呼吸声，说道：“你们不是你们的，你们也不
当前epoch:8
第9轮平均loss:3.809628
让他在半年之前，就不能做出来的事情，但别人都是一个名字，但也是
李慕站在山路上，深深的呼吸口气，说道：“你是什么了，我们的，我
当前epoch:9
第10轮平均loss:3.788001
让他在半年之前，就不能做出来，他们的，他每一个人都是一个小的，
李慕站在山路上，深深的呼吸声道：“我们的，你们不是什么意思？”
当前epoch:10
第11轮平均loss:3.756911
让他在半年之前，就不能做出一个不少，但他们也